### Defining ad-hoc scoring capabilities in Keras

In [ ]:
import sys
import pandas as pd
import numpy as np

from keras import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.utils import to_categorical
from keras import backend as K
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)




    Three classical score Measures (f1, recall, precision)

In [ ]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

    Model definition with non standard measures

In [ ]:
def MLP_model():
   model = Sequential()
   model.add(Dense(128, activation='relu', kernel_initializer='random_normal', input_dim=122))
   model.add(Dense(128, activation='relu', kernel_initializer='random_normal'))
   model.add(Dense(2, activation='sigmoid', kernel_initializer='random_normal'))
   model.compile(optimizer ='adam',loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])
   return model

    Read and prepare data

In [ ]:
data = pd.read_csv("DoS_df.csv")
X_DoS = data.drop('label',1)
y_DoS = data.label

y = np_utils.to_categorical(y_DoS) # t is categorical now
scaler = StandardScaler()
X = scaler.fit_transform(X_DoS)


X_train, X_test, y_train, y_test = train_test_split(
           X, y, test_size=0.20, random_state=42)

    Fit and Score model

In [ ]:
model = MLP_model()

# fit the model
history = model.fit(X_train, y_train, validation_split=0.3, epochs=10, verbose=1)

# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)